In [1]:
# meta=pd.read_json('../Data/TCGA/files.2020-10-21.json')
# meta['cancer']=[meta['cases'].iloc[k][0]['project']['project_id'] for k in range(meta.shape[0])]
# meta=meta.set_index('file_name')['cancer']
# meta.to_csv('../Data/TCGA/metadata_final.txt',sep='\t')
# cat=pd.DataFrame(meta['cancer'].unique()).reset_index()
# cat.columns=['id_category','name_category']
# cat['type_category']='Cancer Tissue'
# cat.to_csv('../Data/TCGA/category.txt',sep='\t',index=False)
# pc=pd.read_csv('../Data/TCGA/gencode.gene.info.v22.tsv',sep='\t')
# pc=pc[pc['gene_type'] == 'protein_coding'][['gene_id','gene_name']].drop_duplicates()
# pc['ensembl']=[i.split('.')[0] for i in pc['gene_id']]
# protein_atlas=pd.read_csv('../../MainLibrary/proteinatlas.tsv',sep='\t',index_col='Ensembl')[['Gene','Gene synonym','Uniprot']]
# pc=pc.merge(protein_atlas,right_on='Ensembl',left_on='ensembl')
# pc.set_index('gene_id').to_csv('../Data/TCGA/protein_coding.txt',sep='\t')

In [1]:
# import pandas as pd
# import os

# pc=pd.read_csv('../Data/TCGA/protein_coding.txt',sep='\t',index_col=0)
# res=pd.DataFrame()
# fin_annot=[]
# path='../Data/TCGA/raw/'
# for i in os.listdir(path):
#     if os.path.isdir(path+i) == False:
#         continue
#     files=os.listdir(path+i)
#     FPKM=[k for k in files if 'FPKM.txt.gz' in k][0]
#     annot=[k for k in files if 'annotation' in k]
#     temp=pd.read_csv('../Data/TCGA/raw/'+i+'/'+FPKM,sep='\t',header=None,index_col=0)[1].reindex(pc.index).fillna(0)
#     temp.name=FPKM
#     res=pd.concat([res,temp],1)
#     fin_annot.append(len(annot))
# res.to_csv('../Data/TCGA/FPKM_TCGA.txt.gz',sep='\t',compression='gzip')
# pd.Series(fin_annot,index=res.columns, name='annotation').to_csv('../Data/TCGA/annot_TCGA.txt',sep='\t')

In [2]:
import pandas as pd
import numpy as np
import os, sys, leidenalg, pickle
from scipy.stats import spearmanr
from statsmodels.sandbox.stats.multicomp import multipletests
import igraph as ig

In [3]:
class Network_Analysis:
    def __init__(self,raw_data,nodes,respath,cat):
        self.res_path=respath
        self.cat=cat
        if os.path.isdir(self.res_path):
            pass
        else:
            os.mkdir(self.res_path)
        self.network_ori=self.__calc(raw_data)
        self.nodes=nodes

        print('Network Analysis')
        self.__net_analysis_combi()
    
    def __calc(self,df):
        print('Calculating Correlation..')
        temp=spearmanr(df.T)
        corr=pd.DataFrame(temp[0],columns=list(df.index),index=list(df.index))
        pval=pd.DataFrame(temp[1],columns=list(df.index),index=list(df.index))
        print('Filtering the matrix Correlation..')
        corr=corr.where(np.triu(np.ones(corr.shape)).astype(np.bool))
        pval=pval.where(np.triu(np.ones(pval.shape)).astype(np.bool))
        print('Making long table of Correlation..')
        corr2=corr.unstack().reset_index(name='weight')
        pval2=pval.unstack().reset_index(name='pval')
        res=corr2.merge(pval2,on=['level_0','level_1'])
        res=res[res['level_0'] != res['level_1']]
        res=res.dropna()
        res=res[['level_0','level_1','weight','pval']]
        res['category'] = self.cat
        res['padj']=multipletests(res['pval'],method='fdr_bh')[1]
        res=res[res.padj < 0.05].reset_index(drop=True)
        res.columns=['source','target','correlation','pvalue','category','padj']
        res=res[['source','target','category','correlation','pvalue','padj']]
        res.to_csv('%s/%d_edges.txt' % (self.res_path,self.cat),sep='\t',index=False)
        print('Done!!')
        return res
    
    def __net_analysis_combi(self):
        print('Loading The Network...')
        temp=self.network_ori
        g= ig.Graph.TupleList(zip(temp['source'],temp['target'],temp['correlation']),weights=True)
        G_pos = g.subgraph_edges(g.es.select(weight_gt = 0), delete_vertices=False)
        G_neg = g.subgraph_edges(g.es.select(weight_lt = 0), delete_vertices=False)
        G_neg.es['weight'] = [-w for w in G_neg.es['weight']]
        part_pos = leidenalg.ModularityVertexPartition(G_pos, weights='weight')
        part_neg = leidenalg.ModularityVertexPartition(G_neg, weights='weight');
        optimiser = leidenalg.Optimiser()
        diff = optimiser.optimise_partition_multiplex([part_pos, part_neg],layer_weights=[1,-1], n_iterations=-1)
        self.clustering_combi=pd.DataFrame(pd.Series(part_pos.membership+part_neg.membership,index=G_pos.vs['name']+G_neg.vs['name'])).reset_index().drop_duplicates().set_index('index')[0]
        print('Cluster Analysis...')
        self.modularity_combi=diff
        open('%s/%d_modularity.txt' % (self.res_path,self.cat),'w').write(str(diff))
        self.nodes['cluster'] = self.clustering_combi.reindex(self.nodes.index).tolist()
        self.nodes.to_csv('%s/%d_nodes.txt' % (self.res_path,self.cat),sep='\t')
    
    def save_network(self):
        pickle_out = open('%s/%d_network_object.pkl' % (self.res_path,self.cat),"wb")
        pickle.dump(self, pickle_out)
        pickle_out.close()

In [4]:
meta=pd.read_csv('../Data/TCGA/metadata_final.txt',sep='\t',index_col=0)
pc=pd.read_csv('../Data/TCGA/protein_coding.txt',sep='\t',index_col=0)
cat=pd.read_csv('../Data/TCGA/category.txt',sep='\t',index_col=0)

In [7]:
i = int(86)
# i= 85
temp=['Unnamed: 0']+list(meta[meta['cancer'] == cat.loc[i]['name_category']].index)
tpm=pd.read_csv('../Data/TCGA/FPKM_TCGA.txt.gz', sep='\t',index_col=0,usecols=temp).dropna(how='all').fillna(0)
tpm=tpm[tpm.mean(1)>5]
nodes=pc[['Gene','ensembl']].reindex(tpm.index)
nodes.columns=['symbol','info1']
mean=tpm.mean(1)
sd=tpm.std(1)
mins=tpm.min(1)
maxs=tpm.max(1)
ran=['%.1f - %.1f' % (mins.loc[k],maxs.loc[k]) for k in mins.index]
mean=['%.1f (+/- %.1f)' % (mean.loc[k],sd.loc[k]) for k in mins.index]
nodes['info2'] = mean
nodes['info3'] =ran
nodes['location'] = 'GENE'
nodes['category'] = i
print('StartingNet')
# k=Network_Analysis(raw_data=tpm,nodes=nodes,respath='../Results/TCGA/',cat=i)
# print('SavingNet')
# k.save_network()
print('Done %d' % i)

StartingNet
Done 86


In [8]:
nodes

,symbol,info1,info2,info3,location,category
ENSG00000000003.13,TSPAN6,ENSG00000000003,9.4 (+/- 8.3),0.1 - 59.1,GENE,86
ENSG00000000419.11,DPM1,ENSG00000000419,37.9 (+/- 18.1),13.0 - 150.4,GENE,86
ENSG00000000971.14,CFH,ENSG00000000971,9.0 (+/- 9.2),0.4 - 69.7,GENE,86
ENSG00000001036.12,FUCA2,ENSG00000001036,29.6 (+/- 14.4),3.9 - 157.2,GENE,86
ENSG00000001084.9,GCLC,ENSG00000001084,6.4 (+/- 3.2),1.1 - 27.0,GENE,86
...,...,...,...,...,...,...
ENSG00000278311.3,GGNBP2,ENSG00000278311,6.2 (+/- 1.8),3.2 - 21.5,GENE,86
ENSG00000278535.3,DHRS11,ENSG00000278535,6.5 (+/- 10.1),0.5 - 117.2,GENE,86
ENSG00000278540.3,ACACA,ENSG00000278540,6.5 (+/- 4.3),1.1 - 59.6,GENE,86
ENSG00000278615.3,C11orf98,ENSG00000278615,5.1 (+/- 2.4),1.1 - 16.3,GENE,86


In [ ]:
x=0
for i in cat.index:
    print('python TCGA.py %d &' % i)
    if (i+1) % 3 == 0:
        print('wait')
print('wait') 